In [ ]:
# ✅ Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Dataset path
DATASET_DIR = '/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X'

# ✅ Imports
import os
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import random

# === CONFIG ===
IMG_SIZE = (224, 224)  # Standard for VGG16
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.0001

# === Visualize Sample Images ===
def plot_sample_images(dataset_path, num_images_per_class=5):
    classes = sorted(os.listdir(dataset_path))
    # plt.figure(figsize=(15, len(classes) * 2.5))
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        sample_images = random.sample(images, min(num_images_per_class, len(images)))
        for i, image_name in enumerate(sample_images):
            img_path = os.path.join(class_path, image_name)
            img = load_img(img_path, target_size=IMG_SIZE)
            # plt.subplot(len(classes), num_images_per_class, class_idx * num_images_per_class + i + 1)
            # plt.imshow(img)
            # plt.axis('off')
            # plt.title(class_name)
    # plt.tight_layout()
    # plt.show()

# === Load Dataset ===
def load_dataset(path):
    images = []
    labels = []
    for label_name in os.listdir(path):
        label_folder = os.path.join(path, label_name)
        if os.path.isdir(label_folder):
            for fname in os.listdir(label_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(label_folder, fname)
                    img = load_img(img_path, target_size=IMG_SIZE)
                    img = img_to_array(img) / 255.0
                    images.append(img)
                    labels.append(label_name)
    return np.array(images), np.array(labels)

# === Load and Prepare Data ===
print("🔄 Loading dataset...")
X, y = load_dataset(DATASET_DIR)
print(f"✅ Loaded {len(X)} images.")

# === Visualize ===
print("🖼 Displaying sample images...")
plot_sample_images(DATASET_DIR)

# === Encode Labels ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
loss_fn = 'binary_crossentropy'
activation_fn = 'sigmoid'
output_units = 1

# === Split Dataset ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === Define Custom VGG16-like Architecture ===
model = Sequential()

# Block 1
model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(*IMG_SIZE, 3)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Block 2
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Block 3
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Block 4
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Block 5
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_units, activation=activation_fn))

# Compile
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=loss_fn, metrics=['accuracy'])
model.summary()

# === Train Model ===
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

# === Evaluate ===
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"📊 Test Accuracy: {test_acc:.2f}")

# === Prediction Function ===
def predict_image(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    label = "Malignant" if prediction[0][0] > 0.5 else "Benign"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    print(f"Prediction: {label} ({confidence:.2f})")
    # plt.imshow(load_img(img_path))
    # plt.title(f"{label} ({confidence:.2f})")
    # plt.axis('off')
    # plt.show()

# Example:
# predict_image('/content/drive/MyDrive/dataset_cancer_v1/classificacao_binaria/100X/benign/img001.png')